# 📊 Задание по методам валидации в машинном обучении  
---  
> урок_№: 1    
домашняя_работа_№: 1  
исполнитель: Маъруф Ибрагимов  

## 1. Загрузка и предобработка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 📥 **Загрузите набор данных** Bank Customer Churn Prediction и обработайте пропуски.

In [2]:
source_file='Bank Customer Churn Prediction.csv'

In [3]:
df=pd.read_csv(source_file)

In [4]:
df.isna().sum()

customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
dtype: int64

In [5]:
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


In [7]:
round(df.describe().T, 2)

,count,mean,std,min,25%,50%,75%,max
customer_id,10000.0,15690940.57,71936.19,15565701.00,15628528.25,15690738.00,15753233.75,15815690.00
credit_score,10000.0,650.53,96.65,350.00,584.00,652.00,718.00,850.00
age,10000.0,38.92,10.49,18.00,32.00,37.00,44.00,92.00
tenure,10000.0,5.01,2.89,0.00,3.00,5.00,7.00,10.00
balance,10000.0,76485.89,62397.41,0.00,0.00,97198.54,127644.24,250898.09
products_number,10000.0,1.53,0.58,1.00,1.00,1.00,2.00,4.00
credit_card,10000.0,0.71,0.46,0.00,0.00,1.00,1.00,1.00
active_member,10000.0,0.52,0.50,0.00,0.00,1.00,1.00,1.00
estimated_salary,10000.0,100090.24,57510.49,11.58,51002.11,100193.92,149388.25,199992.48
churn,10000.0,0.20,0.40,0.00,0.00,0.00,0.00,1.00


In [8]:
df.describe(include='object').T

,count,unique,top,freq
country,10000,3,France,5014
gender,10000,2,Male,5457


In [9]:
df['country'].unique().tolist()

['France', 'Spain', 'Germany']

> Пропусков в файле не обнаружено.

### 🔄 **Разделите данные на признаки (X) и целевую переменную (y).**

In [10]:
df2=df.copy()

In [11]:
target='churn'

X=df2.drop(labels=[target], axis=1)
y=df2[target]

print(X.shape)
print(y.shape)

(10000, 11)
(10000,)


### 📈 **Выведите корреляционную сортированную таблицу** признаков и таргета для определения самых важных признаков, которые влияют на целевую переменную (либо в виде сортированного графика)

In [12]:
corr=df2.select_dtypes(include='number').corr()
target_corr=(
    corr.query("index!=@target")[[target]]
    .assign(abs_corr_coef=lambda d_:d_[target].abs())
    .sort_values(by=['abs_corr_coef'], ascending=False)
)
target_corr

,churn,abs_corr_coef
age,0.285323,0.285323
active_member,-0.156128,0.156128
balance,0.118533,0.118533
products_number,-0.047820,0.047820
credit_score,-0.027094,0.027094
tenure,-0.014001,0.014001
estimated_salary,0.012097,0.012097
credit_card,-0.007138,0.007138
customer_id,-0.006248,0.006248


### 📊 **Выведите распределение целевой переменной в процентном соотношении.**

In [13]:
y.value_counts(normalize=True)

churn
0    0.7963
1    0.2037
Name: proportion, dtype: float64

## 2. Удерживающая проверка (Hold-Out CV)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from catboost import CatBoostClassifier, Pool

### ✂️ **Разделите данные на обучающую и тестовую выборки** в соотношении 80/20.

In [15]:
seed=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=seed)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 11)
(2000, 11)
(8000,)
(2000,)


### 🧠 **Обучите модель** CatBoostClassifier с использованием обучающей выборки.

In [16]:
df2.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
{c:df2[c].nunique() for c in df2.columns}

{'customer_id': 10000,
 'credit_score': 460,
 'country': 3,
 'gender': 2,
 'age': 70,
 'tenure': 11,
 'balance': 6382,
 'products_number': 4,
 'credit_card': 2,
 'active_member': 2,
 'estimated_salary': 9999,
 'churn': 2}

In [18]:
df2.groupby('tenure')[target].value_counts(normalize=True).unstack().sort_index()

churn,0,1
tenure,,
0,0.769976,0.230024
1,0.775845,0.224155
2,0.808206,0.191794
3,0.788900,0.211100
4,0.794742,0.205258
5,0.793478,0.206522
6,0.797311,0.202689
7,0.827821,0.172179
8,0.807805,0.192195


In [19]:
cat_features=['country', 'gender', 'products_number', 'credit_card', 'active_member', 'tenure']

In [20]:
train_pool=Pool(data=X_train, label=y_train, cat_features=cat_features)
test_pool=Pool(data=X_test, label=y_test, cat_features=cat_features)

In [21]:
model=CatBoostClassifier(
    random_state=seed,
    depth=2,
    eval_metric='AUC',
    verbose=0
)

In [22]:
model.fit(train_pool, eval_set=test_pool, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### 📊 **Оцените её производительность** на тестовой выборке, используя метрику AUC.

In [23]:
train_recall=metrics.recall_score(y_train, model.predict(X_train))
train_precision=metrics.precision_score(y_train, model.predict(X_train))
train_accuracy=metrics.accuracy_score(y_train, model.predict(X_train))
train_auc=metrics.roc_auc_score(y_train, model.predict(X_train))

test_recall=metrics.recall_score(y_test, model.predict(X_test))
test_precision=metrics.precision_score(y_test, model.predict(X_test))
test_accuracy=metrics.accuracy_score(y_test, model.predict(X_test))
test_auc=metrics.roc_auc_score(y_test, model.predict(X_test))

In [24]:
print(train_recall, test_recall)
print(train_auc, test_auc)

0.5127737226277372 0.5038167938931297
0.7390803792496773 0.7295064056584504


In [25]:
metrics_labels=['recall', 'precision', 'accuracy', 'roc_auc']
cv_comparison=pd.DataFrame({
        'cv_type':['Hold-Out CV']*8, 
        'metric':metrics_labels*2, 
        'stat':['mean']*4+['std']*4,
        'value':[test_recall, test_precision, test_accuracy, test_auc]+[0]*4
})

In [26]:
cv_comparison

,cv_type,metric,stat,value
0,Hold-Out CV,recall,mean,0.503817
1,Hold-Out CV,precision,mean,0.733333
2,Hold-Out CV,accuracy,mean,0.866500
3,Hold-Out CV,roc_auc,mean,0.729506
4,Hold-Out CV,recall,std,0.000000
5,Hold-Out CV,precision,std,0.000000
6,Hold-Out CV,accuracy,std,0.000000
7,Hold-Out CV,roc_auc,std,0.000000


## 3. k-блочная перекрестная проверка (k-Fold CV)

In [27]:
from sklearn.model_selection import cross_val_score

### 🔁 **Реализуйте k-Fold перекрестную проверку** с использованием 5 блоков.

In [28]:
model_2=CatBoostClassifier(
    random_state=seed,
    depth=2,
    iterations=100,
    eval_metric='AUC',
    verbose=0,
    cat_features=cat_features
)

In [29]:
cv_comparison.loc[len(cv_comparison)]=['test', 's', 'd', 2]

### 📊 **Вычислите среднее значение AUC** для всех блоков.

In [30]:
def compute_n_record(metric, cv_type, cv=10):
    scores = cross_val_score(model_2, X,y, cv=cv, scoring=metric)
    cv_comparison.loc[len(cv_comparison)]=[cv_type, metric, 'mean', scores.mean()]
    cv_comparison.loc[len(cv_comparison)]=[cv_type, metric, 'std', scores.std()]

In [31]:
for label in metrics_labels:
    compute_n_record(metric=label, cv_type='k-Fold CV')

In [32]:
cv_comparison.query("cv_type=='k-Fold CV'")

,cv_type,metric,stat,value
9,k-Fold CV,recall,mean,0.464433
10,k-Fold CV,recall,std,0.041854
11,k-Fold CV,precision,mean,0.765848
12,k-Fold CV,precision,std,0.027599
13,k-Fold CV,accuracy,mean,0.862000
14,k-Fold CV,accuracy,std,0.008989
15,k-Fold CV,roc_auc,mean,0.862983
16,k-Fold CV,roc_auc,std,0.009046


## 4. Стратифицированный k-Fold CV

In [33]:
from sklearn.model_selection import StratifiedKFold

### ⚖️ **Выполните стратифицированную k-Fold перекрестную проверку**

In [34]:
skf=StratifiedKFold(n_splits=10)
for label in metrics_labels:
    compute_n_record(metric=label, cv_type='Stratified k-Fold CV', cv=skf)

### 🔍 **Сравните результаты** со стандартной k-Fold проверкой

In [35]:
(
    cv_comparison
    .query("cv_type.isin(['k-Fold CV', 'Stratified k-Fold CV'])")
    .pivot_table(
        index=['metric'],
        columns=['cv_type', 'stat'],
        values='value',
        aggfunc='mean'
    )
)

cv_type   Stratified k-Fold CV           k-Fold CV          
stat                      mean       std      mean       std
metric                                                      
accuracy              0.862000  0.008989  0.862000  0.008989
precision             0.765848  0.027599  0.765848  0.027599
recall                0.464433  0.041854  0.464433  0.041854
roc_auc               0.862983  0.009046  0.862983  0.009046

### 📝 **Объясните, в чем преимущество стратифицированного подхода.**

## 5. Перекрестная проверка с исключением одного (LOOCV)

In [36]:
from sklearn.model_selection import LeaveOneOut

### 🔄 **Реализуйте метод LOOCV**

In [37]:
loo=LeaveOneOut()

In [79]:
df_sample=df2.groupby(target).sample(frac=.1, random_state=seed)

In [80]:
df_sample

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
8165,15813503,606,Spain,Male,37,8,154712.58,2,1,0,89099.18,0
6409,15690695,683,France,Female,33,9,0.00,2,1,1,38784.42,0
2157,15750649,744,France,Female,44,3,0.00,2,1,1,189016.14,0
1618,15662955,697,France,Male,27,8,141223.68,2,1,0,90591.15,0
1633,15755868,562,France,Male,35,7,0.00,1,0,0,48869.67,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5187,15682421,683,France,Female,30,2,0.00,2,0,1,100496.84,1
2038,15710743,621,France,Male,47,0,0.00,1,1,1,133831.37,1
335,15697441,485,France,Male,29,7,182123.79,1,1,0,116828.51,1
9440,15584156,593,Spain,Male,27,10,0.00,3,0,0,94620.00,1


In [81]:
df_sample['churn'].value_counts(normalize=True)

churn
0    0.796
1    0.204
Name: proportion, dtype: float64

In [82]:
X_sample=df_sample.drop(labels=[target], axis=1)
y_sample=df_sample[target]

In [83]:
y_true=[]
y_pred=[]

In [90]:
for train_ix, test_ix in loo.split(X_sample):
    X_sample_train, X_sample_test = X_sample.iloc[train_ix], X_sample.iloc[test_ix]
    y_sample_train, y_sample_test = y_sample.iloc[train_ix], y_sample.iloc[test_ix]

In [91]:
model_2.fit(X_sample_train, y_sample_train)

In [93]:
model_2.predict(X_sample_test)[0]

1

In [96]:
y_sample_test.iloc[0]

1

In [97]:
for train_ix, test_ix in loo.split(X_sample):
    X_sample_train, X_sample_test = X_sample.iloc[train_ix], X_sample.iloc[test_ix]
    y_sample_train, y_sample_test = y_sample.iloc[train_ix], y_sample.iloc[test_ix]

    model_2.fit(X_sample_train, y_sample_train)

    prediction=model_2.predict(X_sample_test)

    y_pred.append(prediction[0])
    y_true.append(y_sample_test.iloc[0])

    metrics.roc_auc_score(y_true, y_pred)

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [84]:
from datetime import datetime as dtime

In [85]:
for label in metrics_labels:
    time_st=dtime.now().strftime('%H:%M:%S')
    print(f'{time_st} computing for {label}...')
    compute_n_record(metric=label, cv_type='LOO CV', cv=loo)

17:43:02 computing for recall...


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/m

KeyboardInterrupt: 

In [71]:
scores=cross_val_score(model_2, X, y, cv=loo, scoring='roc_auc', n_jobs=-1, verbose=0)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 139, in __call__
    score = scorer._score(
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_ranking.py", line 640, in roc_auc_score
    return _average_binary_score(
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_base.py", line 76, in _average_binary_score
    

/home/codespace/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 139, in __call__
    score = scorer._score(
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_ranking.py", line 640, in roc_auc_score
    return _average_binary_score(
  File "/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_base.py", line 76, in _average_binary_score
    

In [66]:
scores.mean()

nan